# 1. Imports

In [1]:
import pandas as pd
from pathlib import Path
import openpyxl
import string

import re
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

from scipy import sparse
import joblib

DATA_PATH = Path("../data/")

# 2. Load Data

In [6]:
df_nvidia = pd.read_csv(DATA_PATH / "NVIDIA_MergedData_20241101-Present.csv")

df_nvidia['date'] = pd.to_datetime(df_nvidia['date'])
display(df_nvidia.head())

# get daily close prices from df_nvidia, consistent ordering
daily_close = (
    df_nvidia
    .groupby('date')['close']
    .first()
    .sort_index()
)

display(daily_close.head())

,language,sourcecountry,seendate,date,url,title,domain,open,high,low,close,adj_close,volume
0,English,Australia,2024-11-18 03:45:00+00:00,2024-11-18,https://www.fool.com.au/2024/11/18/prediction-...,Prediction : Nvidia stock is going to soar aft...,fool.com.au,139.5,141.55,137.15,140.15,140.11,221205300
1,English,Cyprus,2024-11-18 04:00:00+00:00,2024-11-18,https://cyprus-mail.com/2024/11/18/softbank-fi...,SoftBank first to receive new Nvidia chips for...,cyprus-mail.com,139.5,141.55,137.15,140.15,140.11,221205300
2,English,United States,2024-11-18 06:30:00+00:00,2024-11-18,https://247wallst.com/market-news/2024/11/17/n...,Nasdaq Futures Up Sunday Night : NVIDIA Earnin...,247wallst.com,139.5,141.55,137.15,140.15,140.11,221205300
3,English,United States,2024-11-18 11:00:00+00:00,2024-11-18,https://www.benzinga.com/24/11/42029943/dow-tu...,Dow Tumbles Over 300 Points Following Economic...,benzinga.com,139.5,141.55,137.15,140.15,140.11,221205300
4,English,United States,2024-11-18 11:30:00+00:00,2024-11-18,https://www.fool.com/investing/2024/11/18/nvid...,Prediction : Nvidia Stock Will Soar After Nov ...,fool.com,139.5,141.55,137.15,140.15,140.11,221205300


date
2024-11-18    140.15
2024-11-19    147.01
2024-11-20    145.89
2024-11-21    146.67
2024-11-22    141.95
Name: close, dtype: float64

In [ ]:
X_train_text = sparse.load_npz(f"{DATA_PATH}/tfidf/X_train_tfidf.npz") # Load training TF-IDF matrix
X_test_text  = sparse.load_npz(f"{DATA_PATH}/tfidf/X_test_tfidf.npz") # Load testing TF-IDF matrix
vectorizer   = joblib.load(f"{DATA_PATH}/tfidf/tfidf_vectorizer.pkl") # Load TF-IDF vectorizer